In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn import cross_validation,ensemble,metrics

c:\anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
# читаем массив данных
data = pd.read_csv('lenta_example.csv')
print(len(data))
data.head()

24467


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,url,title,text,topic,tags
0,0,1,1,https://lenta.ru/news/2018/12/15/doping/,Австрия не представила доказательств вины росс...,Австрийские правоохранительные органы не предс...,Спорт,Зимние виды
1,1,5,5,https://lenta.ru/news/2018/12/15/sobor/,Архиепископ канонической УПЦ отказался прийти ...,Архиепископ канонической Украинской православн...,Бывший СССР,Украина
2,2,11,11,https://lenta.ru/news/2018/12/15/ovechkin/,Овечкин повторил свой рекорд,Капитан «Вашингтона» Александр Овечкин сделал...,Спорт,Хоккей
3,3,13,13,https://lenta.ru/news/2018/12/15/newyear/,Названы регионы России с самым дорогим и дешев...,Производитель онлайн-касс «Эвотор» проанализир...,Экономика,Деньги
4,4,14,14,https://lenta.ru/news/2018/12/15/gaz/,Россию и Украину пригласили на переговоры по газу,Вице-президент Еврокомиссии Марош Шефчович при...,Экономика,Госэкономика


In [4]:
# делаем векторизацию - превращаем текст в набор чисел
vectorizer=TfidfVectorizer(analyzer='word')
vectorizer.fit_transform(data['text'] )

<24467x195536 sparse matrix of type '<class 'numpy.float64'>'
	with 3151943 stored elements in Compressed Sparse Row format>

In [9]:
# формируем обучающую выборку
X_train = data.tail(10000)['text']
X_train_ve=vectorizer.fit_transform(X_train)
X_train_arr=X_train_ve.toarray()
y_train = data.tail(10000)['tags']

# формируем тестовую выборку
X_test = data.head(10000)['text']
X_test_ve=vectorizer.transform(X_test)
X_test_arr=X_test_ve.toarray()
y_test = data.head(10000)['tags']
y_test = np.array(y_test)

In [10]:
# обучаем модель
model = LogisticRegression()
model.fit(X_train_arr, y_train)
result = model.predict(X_test_arr)

In [11]:
# посмотрим на результаты
print ('train - ',model.score(X_train_arr, y_train))
print ('test - ',metrics.accuracy_score(y_test,result))
print (metrics.classification_report (y_test,result))
print (metrics.confusion_matrix (y_test,result))

train -  0.8186
test -  0.6939
                  precision    recall  f1-score   support

       undefined       0.48      0.29      0.36       184
            Авто       1.00      0.12      0.21        26
      Белоруссия       0.00      0.00      0.00       115
          Бизнес       0.31      0.51      0.38       256
      Бокс и ММА       0.99      0.93      0.96       379
           Вкусы       0.00      0.00      0.00        20
     Внешний вид       0.75      0.02      0.05       127
             Все       0.00      0.00      0.00        42
         Гаджеты       0.74      0.67      0.70        67
           Город       0.25      0.66      0.37        32
    Госэкономика       0.58      0.91      0.70      1034
            Дача       1.00      0.04      0.07        85
        Движение       0.65      0.33      0.44        39
  Деловой климат       0.75      0.30      0.42       229
          Деньги       0.00      0.00      0.00        96
           Жизнь       0.90      0.20   

c:\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
# попробуем проверить модель вручную
model.predict(vectorizer.transform(['В Омске прошла выставка жареных гвоздей']))

array(['Кино'], dtype=object)

In [ ]:
# аренда жилья превысила спрос 2018 года
# на форуме инноваций Греф презентовал новую платформу для игры на бирже
# В Омске прошла выставка жареных гвоздей
# Трамп наложил новые тарифы на сталь из китая из-за этого было открыто несколько заводов